In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append("../scripts")
# from data_cleaner import Data_cleaner
from util import Util
from data_cleaner import DataCleaner
from plotter import Plotter

In [3]:
util = Util()
cleaner= DataCleaner()
plotter = Plotter()